### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, 
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Загрузка библиотек

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [2]:
#Дополнительно сразу отсечем пустые дни и выделим час из значения времени
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")
#отбираем значение по столбцу «топливный_экв» со значение > 0
energy_0 = energy_0[energy_0["meter_reading"] > 0]
#дублируем столбец "timestamp" и конвертируем в формат «час» 
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())#просмотр первых n строк

      building_id  meter           timestamp  meter_reading  hour
704             0      0 2016-01-30 08:00:00        43.6839     8
725             0      0 2016-01-31 05:00:00        37.5408     5
737             0      0 2016-01-31 17:00:00        52.5571    17
2366            0      0 2016-04-08 14:00:00        59.3827    14
2923            0      0 2016-05-01 19:00:00       448.0000    19


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [3]:
# Выделим 20% всех данных на проверку, остальные оставим на обучение
# используя модуль train_test_split разделяем данные для машинного обучения.
# переменная test_size фактически указывает пропорцию набора тестов – 20%.
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head()) #просмотр первых n строк

      building_id  meter           timestamp  meter_reading  hour
5270            0      0 2016-08-07 14:00:00        302.374    14
4171            0      0 2016-06-22 19:00:00        294.866    19
7358            0      0 2016-11-02 14:00:00        185.656    14
8486            0      0 2016-12-19 14:00:00        189.752    14
4578            0      0 2016-07-09 18:00:00        298.279    18


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [4]:
# Среднее и медианное значение потребление энергии по часам
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

         Среднее   Медиана
hour                      
0     242.004245  240.9440
1     238.470483  240.9440
2     240.068906  243.6740
3     239.105821  244.3570
4     239.722760  244.6980
5     237.031696  245.0390
6     237.874169  243.6740
7     240.721148  246.4040
8     243.323004  239.5790
9     234.193569  234.1180
10    236.038772  238.8960
11    233.238397  240.9440
12    240.072534  243.3325
13    235.714738  244.3570
14    236.759569  244.3570
15    237.833113  242.9910
16    236.429818  242.9910
17    236.203807  240.6025
18    236.448396  238.8960
19    237.800213  237.5310
20    239.032618  237.5310
21    237.714036  237.5310
22    236.629740  236.8480
23    239.785243  240.9440


### Функция проверки модели
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [5]:
#Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: 
#1. с логарифмом значения метрики, 
#2. с предсказанием по среднему, 
#3. с предсказанием по медиане,
#4. с квадратом разницы предсказаний,
#5. с квадратом разницы логарифма значения,
#6. сравнение предсказания с его отсутствием - нулями в значениях.

def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())


      building_id  meter           timestamp  meter_reading  hour  \
8090            0      0 2016-12-03 02:00:00       169.2750     2   
8223            0      0 2016-12-08 15:00:00        70.9863    15   
8244            0      0 2016-12-09 12:00:00        95.5584    12   
7757            0      0 2016-11-19 05:00:00       167.2270     5   
5912            0      0 2016-09-03 08:00:00       238.8960     8   

      meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
8090              0.120873                0.131415             26.393031  
8223              1.438295                1.490000             18.288245  
8244              0.837133                0.861893             20.886253  
7757              0.120471                0.144534             26.268846  
5912              0.000334                0.000008             30.032652  


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [6]:
# Просуммируем квадраты расхождений, разделим на количество значений и извлем квадратный корень:
# вычисляем качество медианы
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
# вычисляем качество среднего
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
# вычисляем качество нуля
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))

print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.26277613671383504
Качество среднего: 0.26087544085073194
Качество нуля: 5.449420712084987
